This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/clutter.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
import open3d as o3d
from pydrake.all import Box, PointCloud, RotationMatrix, StartMeshcat

from manipulation.meshcat_utils import draw_open3d_point_cloud
from manipulation.mustard_depth_camera_example import MustardExampleSystem
from manipulation.open3d_utils import create_open3d_point_cloud


In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Point cloud processing with Open3d

I've produced a scene with multiple cameras looking at our favorite YCB mustard bottle.  I've taken the individual point clouds, converted them into Open3d's point cloud format, estimated their normals, merged the point clouds, cropped then point clouds (to get rid of the geometry from the other cameras), then downsampled the point clouds.  (The order is important!)

I've pushed all of the point clouds to meshcat, but with many of them set to not be visible by default.  Use the drop-down menu to turn them on and off, and make sure you understand basically what is happening on each of the steps.

In [ ]:
def point_cloud_processing_example():
    # This just sets up our mustard bottle with three depth cameras positioned around it.
    system = MustardExampleSystem()

    plant = system.GetSubsystemByName("plant")

    # Evaluate the camera output ports to get the images.
    context = system.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    meshcat.Delete()
    meshcat.SetProperty("/Background", "visible", False)

    pcd = []
    for i in range(3):
        point_cloud = system.GetOutputPort(f"camera{i}_point_cloud").Eval(context)
        meshcat.SetObject(f"pointcloud{i}", point_cloud, point_size=0.001)
        meshcat.SetProperty(f"pointcloud{i}", "visible", False)

        cloud = create_open3d_point_cloud(point_cloud)
        # Crop to region of interest.
        pcd.append(cloud.crop(
            o3d.geometry.AxisAlignedBoundingBox(min_bound=[-.3, -.3, -.3],
                                                max_bound=[.3, .3, .3])))
        draw_open3d_point_cloud(meshcat, f"pointcloud{i}_cropped", pcd[i])
        meshcat.SetProperty(f"pointcloud{i}_cropped", "visible", False)

        pcd[i].estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(f"camera{i}")
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd[i].orient_normals_towards_camera_location(X_C.translation())

    # Merge point clouds.  (Note: You might need something more clever here for
    # noisier point clouds; but this can often work!)
    merged_pcd = pcd[0] + pcd[1] + pcd[2]
    draw_open3d_point_cloud(meshcat, "merged", merged_pcd)

    # Voxelize down-sample.  (Note that the normals still look reasonable)
    down_sampled_pcd = merged_pcd.voxel_down_sample(voxel_size=0.005)
    draw_open3d_point_cloud(meshcat, "down_sampled", down_sampled_pcd)

    # TODO(russt): Make normal rendering work in Meshcat C++
    #draw_open3d_point_cloud(v["normals"], down_sampled_pcd, normals_scale=0.01)
    ## Let the normals be drawn, only turn off the object...
    #v["normals"]["<object>"].set_property("visible", False)

    # If we wanted to show it in the open3d visualizer, we would use...
    # print("Use 'n' to show normals, and '+/-' to change their size.")
    # o3d.visualization.draw_geometries([down_sampled_pcd])


point_cloud_processing_example()